In [300]:
import pulp as pl
from gurobipy import Model, itertools, GRB
from collections import OrderedDict
import os

In [301]:
class Activity():
    ARRIVAL = "Arrival"
    DEPARTURE = "Departure"

class Station():
    def __init__(self, name, data) -> None:
        self.name = name
        self.data = data
    
class Edge(object):
    def __init__(self, nDEP, nARR, lb, ub) -> None:
        self.nodeDEP = nDEP
        self.nodeARR = nARR
        self.lb = lb
        self.ub = ub

    def __hash__(self) -> int:
        return hash((self.nodeDEP, self.nodeARR, self.lb, self.ub))
    
    def __eq__(self, other: object) -> bool:
        return (self.nodeDEP, self.nodeARR, self.lb, self.ub) == (other.nodeDEP, other.nodeARR, other.lb, other.ub)
    
    def __ne__(self, other: object) -> bool:
        return not (self == other)
    
    def __str__(self) -> str:
        return "(%s,%s,%s,%s)" % (self.nodeDEP, self.nodeARR, self.lb, self.ub)
    
class Node(object):
    def __init__(self, line, activity, station) -> None:
        self.line = line
        self.activity = activity
        self.station = station
    
    def __hash__(self) -> int:
        return hash((self.line, self.activity, self.station))
    
    def __eq__(self, other: object) -> bool:
        return (self.line, self.activity, self.station) == (other.line, other.activity, other.station)
    
    def __ne__(self, other: object) -> bool:
        return not self == other
    
    def __str__(self) -> str:
        return "(%s,%s,%s)" % (self.line, self.activity, self.station)

class Graph(object):
    def __init__(self):
        # Graph-Objekt
        self._graph_dict = OrderedDict()

    def nodes(self):
        # Alle Knoten des Graphen
        return list(self._graph_dict.keys())

    def edges(self):
        # Alle Kanten eines Graphen
        return self.init_edges()

    def add_node(self, node):
        # Wenn Knoten nicht existiert
        if node not in self._graph_dict:
            # Fuege Knoten mit leerer Liste hinzu
            self._graph_dict[node] = []

    def add_edge(self, edge):
        # Wenn Vertex schon in Graph ist
        if edge.nodeDEP in self._graph_dict:
            # Haenge Kante an Vertex
            self._graph_dict[edge.nodeDEP].append(edge)
        else:
            # Fuege Kante an Vertex
            self._graph_dict[edge.nodeDEP] = [edge]

    def init_edges(self):
        edges = []
        for node in self._graph_dict:
            for neighbour_edges in self._graph_dict[node]:
                if isinstance(neighbour_edges, Edge):
                    neighbour_edges = [neighbour_edges]
                for neighbour_edge in neighbour_edges:
                    assert isinstance(neighbour_edge, Edge)
                    # if neighbour_edge not in edges:
                    edges.append(neighbour_edge)
        return edges

    def __str__(self):
        res = "Nodes: "
        for k in self._graph_dict:
            res += str(k) + " "
        res += "\nEdges: "
        for edge in self.init_edges():
            res += str(edge) + " "
        return res

    @property
    def graph_dict(self):
        return self._graph_dict

### Linien-Daten

In [302]:
lines = OrderedDict()

#### RE6

##### Stationen

In [303]:
re6_stations = OrderedDict()
stops = open(os.getcwd() + "/Lines/RE6/Stations/RE6 Fahrplan.txt").readlines()
stops = [x.replace('\n', '') for x in stops]

for station in stops:
    re6_stations[station] = None

##### Haltezeit

Alle Haltezeiten auslesen

In [304]:
all_times = []
times = open(os.getcwd() + "/Lines/RE6/Stops/RE6 Haltezeit.txt").readlines()
for line in times:
    lb, ub = line.strip().split(",")
    all_times.append((int(lb),int(ub)))

Haltezeiten zu Station zuordnen

In [305]:
for i, station in enumerate(re6_stations):
    re6_stations[station] = all_times[i]

##### Fahrzeit

Alle Fahrzeiten auslesen

In [306]:
all_times = []
times = open(os.getcwd() + "/Lines/RE6/Drives/RE6 Fahrzeit.txt").readlines()
for line in times:
    lb, ub = line.strip().split(",")
    all_times.append((int(lb),int(ub)))

Streckenpaare erstellen

In [307]:
pairs = []
prev = None
for station in re6_stations:
  if prev is not None:
    pair = (prev, station)
    pairs.append(pair)
  prev = station

Fahrzeit zu Strecken ordnen

In [308]:
re6_drives = OrderedDict()
for i, pair in enumerate(pairs):
    re6_drives[pair] = all_times[i]

#### RB69

##### Stationen

In [309]:
rb69_stations = OrderedDict()
stops = open(os.getcwd() + "/Lines/RB69/Stations/RB69 Fahrplan.txt").readlines()
stops = [x.replace('\n', '') for x in stops]

for station in stops:
    rb69_stations[station] = None

##### Haltezeit

Alle Haltezeiten auslesen

In [310]:
all_times = []
times = open(os.getcwd() + "/Lines/RB69/Stops/RB69 Haltezeit.txt").readlines()
for line in times:
    lb, ub = line.strip().split(",")
    all_times.append((int(lb),int(ub)))

Haltezeiten zu Stationen zuordnen

In [311]:
for i, station in enumerate(rb69_stations):
    rb69_stations[station] = all_times[i]

##### Fahrzeit

Alle Fahrzeiten auslesen

In [312]:
all_times = []
times = open(os.getcwd() + "/Lines/RB69/Drives/RB69 Fahrzeit.txt").readlines()
for line in times:
    lb, ub = line.strip().split(",")
    all_times.append((int(lb),int(ub)))

Streckenpaare erstellen

In [313]:
pairs = []
prev = None
for station in rb69_stations:
  if prev is not None:
    pair = (prev, station)
    pairs.append(pair)
  prev = station

Fahrzeit zu Strecken zuordnen

In [314]:
rb69_drives = OrderedDict()
for i, pair in enumerate(pairs):
    rb69_drives[pair] = all_times[i]

#### RB67

##### Stationen

In [315]:
rb67_stations = OrderedDict()
stops = open(os.getcwd() + "/Lines/RB67/Stations/RB67 Fahrplan.txt").readlines()
stops = [x.replace('\n', '') for x in stops]

for station in stops:
    rb67_stations[station] = None

##### Haltezeit

Alle Haltezeiten auslesen

In [316]:
all_times = []
times = open(os.getcwd() + "/Lines/RB67/Stops/RB67 Haltezeit.txt").readlines()
for line in times:
    lb, ub = line.strip().split(",")
    all_times.append((int(lb),int(ub)))

Haltezeiten zu Stationen zuordnen

In [317]:
for i, station in enumerate(rb67_stations):
    rb67_stations[station] = all_times[i]

##### Fahrzeit

Alle Fahrzeiten auslesen

In [318]:
all_times = []
times = open(os.getcwd() + "/Lines/RB67/Drives/RB67 Fahrzeit.txt").readlines()
for line in times:
    lb, ub = line.strip().split(",")
    all_times.append((int(lb),int(ub)))

Streckenpaare erstellen

In [319]:
pairs = []
prev = None
for station in rb67_stations:
  if prev is not None:
    pair = (prev, station)
    pairs.append(pair)
  prev = station

Fahrzeit zu Strecken zuordnen

In [320]:
rb67_drives = OrderedDict()
for i, pair in enumerate(pairs):
    rb67_drives[pair] = all_times[i]

#### Linien zusammefassen

In [321]:
lines["RE6"] = (re6_stations, re6_drives)
lines["RB69"] = (rb69_stations, rb69_drives)
lines["RB67"] = (rb67_stations, rb67_drives)

LINES_DATA = lines

### Parameter festlegen

In [322]:
T = 60 # 60min

In [323]:
print(rb67_drives)

OrderedDict([(('Bielefeld Hbf', 'BI-Brackwede'), (4, 6)), (('BI-Brackwede', 'Isselhorst-Avenwedde'), (3, 5)), (('Isselhorst-Avenwedde', 'Guetersloh Hbf'), (3, 5)), (('Guetersloh Hbf', 'Rheda-Wiedenbrueck'), (4, 6)), (('Rheda-Wiedenbrueck', 'Herzebrock'), (5, 8)), (('Herzebrock', 'Clarholz'), (3, 5)), (('Clarholz', 'Beelen'), (7, 10)), (('Beelen', 'Warendorf'), (6, 9)), (('Warendorf', 'Warendorf-Einen-Muessingen'), (7, 10)), (('Warendorf-Einen-Muessingen', 'Telgte'), (9, 13)), (('Telgte', 'Muenster (Westf) Hbf'), (11, 15))])


### Erstellen der Graphen

In [324]:
graphs = []
for line, data in LINES_DATA.items():
    start = True
    prev_station = None
    ff_station = None
    prev_nodeARR = None
    graph = Graph()
    LINE_STATIONS = data[0]
    LINE_DRIVES = data[1] 
    for station in LINE_STATIONS:
        if not start:
            next_station = station
            nodeDEP = Node(line, Activity.DEPARTURE, prev_station)
            nodeARR = Node(line, Activity.ARRIVAL, next_station)
            edge = Edge(nodeDEP, nodeARR, LINE_DRIVES[(prev_station, next_station)][0], LINE_DRIVES[(prev_station, next_station)][1])
            graph.add_node(nodeDEP)
            graph.add_node(nodeARR)
            graph.add_edge(edge)
            prev_station = next_station

            if prev_nodeARR is not None:
                prev_edge = Edge(prev_nodeARR, nodeDEP, LINE_STATIONS[station][0], LINE_STATIONS[station][1])
                graph.add_edge(prev_edge)
                prev_nodeARR = nodeARR
            else:
                prev_nodeARR = nodeARR
                continue
        else:
            start = False
            prev_station = station
        
    graphs.append(graph)

In [325]:
for g in graphs:
    print(g)

Nodes: (RE6,Departure,Minden (Westf)) (RE6,Arrival,Porta Westfalica) (RE6,Departure,Porta Westfalica) (RE6,Arrival,Bad Oeynhausen) (RE6,Departure,Bad Oeynhausen) (RE6,Arrival,Loehne (Westf)) (RE6,Departure,Loehne (Westf)) (RE6,Arrival,Herford) (RE6,Departure,Herford) (RE6,Arrival,Bielefeld Hbf) (RE6,Departure,Bielefeld Hbf) (RE6,Arrival,Guetersloh Hbf) (RE6,Departure,Guetersloh Hbf) (RE6,Arrival,Rheda-Wiedenbrueck) (RE6,Departure,Rheda-Wiedenbrueck) (RE6,Arrival,Oelde) (RE6,Departure,Oelde) (RE6,Arrival,Beckum-Neubeckum) (RE6,Departure,Beckum-Neubeckum) (RE6,Arrival,Ahlen (Westf)) (RE6,Departure,Ahlen (Westf)) (RE6,Arrival,Hamm-Heessen) (RE6,Departure,Hamm-Heessen) (RE6,Arrival,Hamm (Westf) Hbf) (RE6,Departure,Hamm (Westf) Hbf) (RE6,Arrival,Kamen) (RE6,Departure,Kamen) (RE6,Arrival,Dortmund Hbf) (RE6,Departure,Dortmund Hbf) (RE6,Arrival,Bochum Hbf) (RE6,Departure,Bochum Hbf) (RE6,Arrival,Wattenscheid) (RE6,Departure,Wattenscheid) (RE6,Arrival,Essen Hbf) (RE6,Departure,Essen Hbf) (RE6,A

In [326]:
#def generate_bb_model(self):
"""
:type graphs: list of LineGraph
"""
bb_model = Model("PESP")
j = 0
objective_function = 0
dict_of_variables = {}
for graph in graphs:
    vertices = graph.nodes()
    edges = graph.edges()

    for edge in edges:
        vertex1 = edge.nodeDEP
        vertex2 = edge.nodeARR
        lin_expr_for_edge = 0
        p = bb_model.addVar(vtype=GRB.INTEGER, name="P_" + str(j))
        bb_model.update()
        bb_model.addConstr(p <= 2, "CP_%s_1" % str(j))
        bb_model.addConstr(p >= 0, "CP_%s_2" % str(j))

        for vertex in vertices:
            if vertex in dict_of_variables.keys():
                t = dict_of_variables[vertex]
            else:
                t = bb_model.addVar(vtype=GRB.INTEGER, name=str(vertex))
                dict_of_variables[vertex] = t
                bb_model.update()
                bb_model.addConstr(t <= T - 1, "CT_%s_1" % str(j))
                bb_model.addConstr(t >= 0, "CT_%s_2" % str(j))

            val = 0
            if vertex == vertex1:
                val = -1
            elif vertex == vertex2:
                val = 1

            lin_expr_for_edge += val * t

        bb_model.addConstr(lin_expr_for_edge + p * T <= edge.ub, "CE_%s_1" % str(j))
        bb_model.addConstr(lin_expr_for_edge + p * T >= edge.lb, "CE_%s_2" % str(j))
        objective_function += lin_expr_for_edge + p * T - edge.lb
        j += 1

    # between lines we need to add nes constrains
    for pair in itertools.combinations(graphs, r=2):
        graph1 = pair[0]
        graph2 = pair[1]

        j = 1
        for vert1 in graph1.nodes():
            for vert2 in graph2.nodes():
                if vert1.station == vert2.station and vert1.activity == "Arrival" and vert2.activity == 'Departure':
                    t1 = dict_of_variables[vert1]
                    t2 = dict_of_variables[vert2]
                    p = bb_model.addVar(vtype=GRB.INTEGER, name="PB_" + str(j))
                    bb_model.update()
                    bb_model.addConstr(p <= 2, "CPE_%s_1" % str(j))
                    bb_model.addConstr(p >= 0, "CPE_%s_2" % str(j))
                    # >= and <=
                    bb_model.addConstr(t2 - t1 + p * T <= ub, "CB_%s_1" % str(j))
                    bb_model.addConstr(t2 - t1 + p * T >= lb, "C_%s_2" % str(j))
                    objective_function += t2 - t1 + p * T - lb
                    j += 1

    bb_model.update()
    bb_model.setObjective(objective_function, GRB.MINIMIZE)
    bb_model.optimize()





KeyError: <__main__.Node object at 0x114acfeb0>